In [1]:
from utils import css_from_file
css_from_file('style/style.css')

Data preprocessing
----------------------

Data preprocessing is the most important step in the model preparation. It takes 90% of the time to prepare and clean the data so it can be processed by a predictive algorithm.

Here we have the data from Rossmann competition https://www.kaggle.com/c/rossmann-store-sales.

It is a good example of a dataset with many different types of data.

In [2]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
training_data = pd.read_csv("data/rossmann/train.csv")
store_data = pd.read_csv("data/rossmann/store.csv")

There are information about the Sales (our target).

In [4]:
print("Training data shape", training_data.shape)
training_data.head()

Training data shape (1017209, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


And the stores themselves

In [5]:
print("Store data shape", store_data.shape)
store_data.head()

Store data shape (1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


Let's join the data

In [6]:
combined_data = pd.merge(training_data, store_data, on="Store")

# sample the data
combined_data = combined_data.sample(frac=0.1).reset_index()

print("Combined data shape", combined_data.shape)
combined_data.head()

Combined data shape (101721, 19)


,index,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,896843,984,2,2014-10-07,8452,887,1,1,0,1,c,a,440.0,NaN,NaN,1,1.0,2013.0,"Jan,Apr,Jul,Oct"
1,572089,628,1,2015-06-22,5300,743,1,0,0,0,a,c,2180.0,NaN,NaN,0,NaN,NaN,NaN
2,621336,682,7,2013-05-12,7489,1273,1,0,0,0,b,a,150.0,9.0,2006.0,0,NaN,NaN,NaN
3,704154,773,2,2013-04-02,6674,954,1,0,0,1,a,a,200.0,NaN,NaN,0,NaN,NaN,NaN
4,492753,541,6,2014-11-08,6189,823,1,0,0,0,a,c,650.0,7.0,1990.0,0,NaN,NaN,NaN


In [7]:
combined_data.PromoInterval.value_counts()

Jan,Apr,Jul,Oct     29167
Feb,May,Aug,Nov     11932
Mar,Jun,Sept,Dec     9808
Name: PromoInterval, dtype: int64

Exercise
----------------------

1. Identify types of data present in the dataset:
    - what would you do with each type of data?
    - are there missing values?
2. Write transformer `PandasSelector` which can select subsets of columns from the dataset.
3. Write transformers for each type of data that convert selected columns to numerical values.
4. Combine it all into 1 pipeline using `make_pipeline` and `make_union` functions.

Hint: you will need those imports
```python
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
```

Explanation:
`BaseEstimator` and `TransformerMixin` are the classes from which you need to inherit in the your transformer class. They are needed for proper pipeline serialization (saving).

`DictVectorizer` is a transformer that can create a matrix from a dictionary of values - it is helpful to convert categorical variables. 

For example:
Let's say you have 2 columns which you want to convert to a matrix: `StoreType` and `Assortment`

DO NOT USE pandas.get_dummies to create binary features
-----------------

In [8]:
# let's convert 2 columns to a list of dictionaries
data_as_dict = list(store_data.loc[:, ["StoreType","Assortment"]].to_dict(orient='records'))[:10]
data_as_dict
# store_data.to_dict()

[{'Assortment': 'a', 'StoreType': 'c'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'c', 'StoreType': 'c'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'c', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'c', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'}]

In [9]:
from sklearn.feature_extraction import DictVectorizer
categorical_transformer = DictVectorizer()
print(categorical_transformer.fit_transform(data_as_dict).todense()) # by default DictVectorizer returns sparse matrix

[[1. 0. 0. 1.]
 [1. 0. 1. 0.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 1. 0.]
 [1. 0. 1. 0.]
 [0. 1. 1. 0.]
 [1. 0. 1. 0.]
 [0. 1. 1. 0.]
 [1. 0. 1. 0.]]


In [10]:
from sklearn.feature_extraction import DictVectorizer
data_as_dict[0]['Assortment'] = 'b'
print(categorical_transformer.transform(data_as_dict).todense()) # by default DictVectorizer returns sparse matrix

[[0. 0. 0. 1.]
 [1. 0. 1. 0.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 1. 0.]
 [1. 0. 1. 0.]
 [0. 1. 1. 0.]
 [1. 0. 1. 0.]
 [0. 1. 1. 0.]
 [1. 0. 1. 0.]]


Alternative way (or maybe the main way)
------------------

Use a Python package that does it for you. http://contrib.scikit-learn.org/categorical-encoding/index.html

In [25]:
# !pip install category_encoders
from category_encoders.one_hot import OneHotEncoder
one_hot = OneHotEncoder()
# one_hot.fit_transform(store_data[["StoreType","Assortment"]].values)

Exercise template
-----------------

Your final process should like like this:
    
```python
from sklearn.preprocessing import Imputer

processing_pipeline = make_pipeline(
    # combine features
    make_union(
        make_pipeline(
            PandasSelector(["StoreType","Assortment"]),
            ? (.T.to_dict().values() called),
            DictVectorizer(),
            # select categorical data
            # do something with categorical data
        ),
        make_pipeline(
            PandasSelector(["Date"]),
            TransformDates(),
            # select date
            # do something with dates
            # first convert text to date using pd.to_datetime
            # use .dt attribute of pandas column
        ),
        make_pipeline(
            PandasSelector([list of numerical columns]),
            Imputer(),
            # select numerical data
            # do something with numerical data
        ),
        make_pipeline(
            # make some feature engineering
        )
    )
)
```

In [16]:
import pandas_profiling

report = pandas_profiling.ProfileReport(store_data)

# report

In [67]:
test = combined_data.loc[:, 'Date']

# print(test.loc[:,'Date'])
dates =  pd.to_datetime(test)

# pd.DataFrame({'year': dates.dt.year, 'month': dates.dt.month, 'day': dates.dt.day})
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101721 entries, 0 to 101720
Data columns (total 19 columns):
index                        101721 non-null int64
Store                        101721 non-null int64
DayOfWeek                    101721 non-null int64
Date                         101721 non-null object
Sales                        101721 non-null int64
Customers                    101721 non-null int64
Open                         101721 non-null int64
Promo                        101721 non-null int64
StateHoliday                 101721 non-null object
SchoolHoliday                101721 non-null int64
StoreType                    101721 non-null object
Assortment                   101721 non-null object
CompetitionDistance          101440 non-null float64
CompetitionOpenSinceMonth    69424 non-null float64
CompetitionOpenSinceYear     69424 non-null float64
Promo2                       101721 non-null int64
Promo2SinceWeek              50907 non-null float64
Promo2SinceYe

In [49]:
# combined_data.select_dtypes(['float64', 'int64']).head()#.columns.values

In [125]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer

# import pdb
# pdb.set_trace()

class PandasSelector(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols

    def fit(self, x, y = None):
        return self

    def transform(self, x):
        return x.loc[:, self.cols]

class TransformDates(BaseEstimator, TransformerMixin):
    def __init(self):
        pass
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        dates = pd.to_datetime(x.iloc[:,0])
        return pd.DataFrame({'year': dates.dt.year, 
                             'month': dates.dt.month, 
                             'day': dates.dt.day
                            })
    
class PandasSelectType(BaseEstimator, TransformerMixin):
    def __init__(self, dtypes_list):
        self.dtypes_list = dtypes_list
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return x.select_dtypes(self.dtypes_list)
    
    
    
processing_pipeline = make_pipeline(
    # combine features
    make_union(
        make_pipeline(
            PandasSelector(['StoreType', 'Assortment']),
            OneHotEncoder()
        ),
        make_pipeline(
            PandasSelector(['Date']),
            TransformDates()
        ),
        make_pipeline(
            PandasSelectType(['float64', 'int64']),
            Imputer(strategy='mean')
        )
    )
)


In [126]:
# processing_pipeline.fit_transform(combined_data)
# processing_pipeline.fit(combined_data)
# preprocessed_data = processing_pipeline.fit_transform(combined_data)
# pd.DataFrame(preprocessed_data)

Pipeline(memory=None,
     steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(memory=None,
     steps=[('pandasselector', PandasSelector(cols=['StoreType', 'Assortment'])), ('onehotencoder', OneHotEncoder(cols=['StoreType', 'Assortment'], drop_invariant=False,
       handle_unknown='impute', impute_missing=True, return_df=True,
      ...4'])), ('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0))]))],
       transformer_weights=None))])

**Double click to see the solution**

<div class='spoiler'>

class PandasSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x.loc[:,self.columns]
    
    
class PandasToDict(BaseEstimator, TransformerMixin):

    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x.T.to_dict().values()

    
class ExtractDateAttributes(BaseEstimator, TransformerMixin):
    
    def __init__(self, date_format=None,
                 attributes=["year","month","day","weekday"]):
        self.date_format = date_format
        self.attributes = attributes
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        assert x.shape[1] == 1, "This transformer can handle 1 date"
        
        # convert data to date
        dt = pd.to_datetime(x.ix[:,0])
        
        # create an empty DataFrame
        df = pd.DataFrame()
        
        for attr in self.attributes:
            df[attr] = getattr(dt.dt, attr)
            
        return df
    

processing_pipeline = make_pipeline(
    # Select used variables
    PandasSelector(["Open", "Promo", "SchoolHoliday", 
                    "Date", "StoreType", "Assortment",
                    "CompetitionDistance", "CompetitionOpenSinceMonth",
                    "CompetitionOpenSinceYear", "Promo2",
                    "Promo2SinceWeek", "Promo2SinceYear"]),
    
    # combine features
    make_union(
        make_pipeline(
            PandasSelector(["Open", "Promo", "SchoolHoliday", "CompetitionDistance", 
                            "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", 
                            "Promo2", "Promo2SinceWeek", "Promo2SinceYear"]),
            Imputer(strategy='mean')
        ),
        make_pipeline(
            PandasSelector(["Date"]),
            ExtractDateAttributes()
        ),
        make_pipeline(
            PandasSelector(["StoreType", "Assortment"]),
            PandasToDict(),
            DictVectorizer(sparse=False)
        )
    )
)

</div>

To combine together your data processing pipeline and predictive algorithm you can chain them using `make_pipeline` function.

## Exercise

1. **Run and understand the code below**
   - explain each line <br/><br/>
   
2. **Why the CV is so low?**
   - the problem is to predict future prices is cross validation a good method to check if models learns well? <br/><br/>
   
3. **Use different splitting scheme and compare results** 
   - train on dates < X and test on dates > X, where X is some date
   
4. **Plot the errors of the predictions depending on how far ahead you make the prediction**

In [93]:
cols = list(combined_data.columns)
cols.remove('Sales')
# cols

In [127]:
from sklearn.cross_validation import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from metrics import rmspe

est = RandomForestRegressor(verbose=True, n_jobs=-1)

pred = cross_val_predict(make_pipeline(processing_pipeline, est), 
                         combined_data.loc[:,cols], 
                         np.log1p(combined_data.loc[:,'Sales']),
                         cv=5)

print(rmspe(combined_data.Sales, np.expm1(pred)))

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


0.08866515540938112


Click here to see the solution

<div class="spoiler">

from cross_validation import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from metrics import rmspe

est = RandomForestRegressor(verbose=True, n_jobs=-1)
training_data = combined_data[combined_data.Date <= '2015-05-30']
test_data = combined_data[combined_data.Date > '2015-05-30']

model = make_pipeline(processing_pipeline, est)

model.fit(training_data, np.log1p(training_data.Sales))

predictions = model.predict(test_data)

print(rmspe(combined_data.Sales, np.expm1(pred)))

</div>

In [129]:
from sklearn.cross_validation import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from metrics import rmspe

est = RandomForestRegressor(verbose=True, n_jobs=-1)
training_data = combined_data[combined_data.Date <= '2015-05-30']
test_data = combined_data[combined_data.Date > '2015-05-30']

model = make_pipeline(processing_pipeline, est)

processing_pipeline.fit(training_data.loc[:,cols])

model.fit(training_data.loc[:,cols], y=np.log1p(training_data.loc[:,'Sales']))

predictions = model.predict(test_data.loc[:,cols])

print(rmspe(combined_data.loc[:,'Sales'], np.expm1(pred)))

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.5s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


0.08866515540938112


In [111]:
training_data.loc[:,cols].head()

,index,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,896843,984,2,2014-10-07,887,1,1,0,1,c,a,440.0,NaN,NaN,1,1.0,2013.0,"Jan,Apr,Jul,Oct"
2,621336,682,7,2013-05-12,1273,1,0,0,0,b,a,150.0,9.0,2006.0,0,NaN,NaN,NaN
3,704154,773,2,2013-04-02,954,1,0,0,1,a,a,200.0,NaN,NaN,0,NaN,NaN,NaN
4,492753,541,6,2014-11-08,823,1,0,0,0,a,c,650.0,7.0,1990.0,0,NaN,NaN,NaN
5,805224,883,4,2013-03-07,566,1,1,0,0,a,a,3200.0,6.0,2005.0,0,NaN,NaN,NaN
